## LABORATORY 03: MACHINE LEARNING I - REGRESSION PROBLEM

### Case of Study 1: Predicting Melbourne Housing Prices

#### 1. Load the dataset

In [1]:
#import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbn

In [ ]:
# load dataset
dataset = pd.read_csv('dataset/melbourne_housing.csv', sep = ',')
metadata = dataset.columns
dataset.head()

In [ ]:
# dimensions of dataset
print("#samples = ", dataset.shape[0])
print("#features = ", dataset.shape[1])

In [4]:
# manage metadata
def get_metadata(data):
    metadata = data.columns
    numerical_cols = data.select_dtypes(include = ["float64", "int64"]).columns.tolist()
    categorical_cols = data.select_dtypes(include = ["object"]).columns.tolist()
    print("Numerical features: ", numerical_cols)
    print("Categorical features: ", categorical_cols)
    return metadata, numerical_cols, categorical_cols

In [ ]:
metadata, numeric_cols, categ_cols = get_metadata(dataset)

#### 2. Exploratory Data Analysis (EDA)

Filtering missing values

In [6]:
# function to filter missing data
def filter_missing(data):
    sbn.displot(
        data = data.isna().melt(value_name="missing"),
        y = "variable",
        hue = "missing",
        multiple = "fill",
        aspect = 1.5
    )

    plt.show()

In [ ]:
# original state of missing values
filter_missing(dataset)

Numerical Features

In [8]:
# function to plot histogram of frequencies
def hist_frequencies(data, numeric_cols, bins):
    # calculate the nrows and ncols for plots
    ncol_plots = 3
    nrow_plots = (len(numeric_cols) + ncol_plots - 1) // ncol_plots
    # create the subplots for specific row and column
    fig, axs = plt.subplots(nrow_plots, ncol_plots, figsize = (16, 4 * nrow_plots))
    axs = axs.flatten()

    for i, col in enumerate(numeric_cols):
        sbn.histplot(data[col], color = "blue", bins = bins, ax = axs[i])
        axs[i].set_title("Histogram of frequencies for " + col)
        plt.xlabel(col)
        plt.ylabel("Frequencies")
    plt.tight_layout()
    plt.show()

In [ ]:
hist_frequencies(dataset, numeric_cols, bins = 10)

In [ ]:
# statistical metrics
display(dataset[numeric_cols].describe())

Categorical Features

In [ ]:
# cycle to calculate number of instances in each categorical column
for col in categ_cols:
    print("\n***** " + col + " ******")
    print(dataset[col].value_counts())

#### 3. Data Preparation

Feature Engineer

In [12]:
# drop the columns which don't have any relevance
def feature_engineer(data):
    # extract information from Date    
    data["Date"] = pd.to_datetime(data["Date"], errors = "coerce")
    data["SaleDayOfWeek"] = data["Date"].dt.dayofweek
    
    # drop non-relevant columns 
    nrelev_cols = ["Address", "BuildingArea", "YearBuilt", "Date", "SellerG"]
    data = data.drop(nrelev_cols, axis = 1)

    return data

In [13]:
dataset = feature_engineer(dataset)

In [ ]:
print("#samples = ", dataset.shape[0])
print("#features = ", dataset.shape[1])

In [ ]:
metadata, numeric_cols, categ_cols = get_metadata(dataset)

In [ ]:
filter_missing(dataset)

Data imputation with K-Nearest Neighbors (KNN)

In [17]:
from sklearn.impute import KNNImputer

def imputation_data(data, num_cols, categ_cols):
    # imputation for numerical columns
    knn_imputer = KNNImputer(n_neighbors = 5)
    data[num_cols] = knn_imputer.fit_transform(data[num_cols])
    # imputation for categorical columns
    for col in categ_cols:
        data[col] = data[col].fillna(data[col].mode()[0])
    
    return data

In [ ]:
data = imputation_data(dataset, numeric_cols, categ_cols)
data.head()

In [ ]:
# state of missing values after imputation
filter_missing(dataset)

Exploration of Data after Imputation

In [23]:
# check type of relationship between variables
def gen_pairplot(data, metadata):
    sbn.set_theme(context = 'notebook', style = 'darkgrid')    
    sbn.pairplot(data[metadata], height = 2.0)
    plt.show()

In [ ]:
gen_pairplot(dataset, numeric_cols)

In [112]:
# function to plot correlation between numerical variables
def plot_correlation(data, cols):
    corr = data[cols].corr()
    plt.matshow(corr, cmap = "coolwarm")
    plt.xticks(range(len(cols)), cols, rotation = 90)
    plt.yticks(range(len(cols)), cols)

    # add the correlation values in each cell
    for (i, j), val in np.ndenumerate(corr):
        plt.text(j, i, f"{val:.1f}", ha='center', va='center', color='black')
    plt.title("Correlation Analysis")
    plt.colorbar()    
    plt.show()

In [ ]:
plot_correlation(dataset, numeric_cols)

In [ ]:
# probability distribution of the target variable
plt.figure(figsize=[8,4])
sbn.histplot(dataset["Price"], color='g', edgecolor="black", linewidth=2, bins=20)

plt.title("Target Variable Distribution - Median Value of Homes ($1Ms)")
plt.show()

#### 4. Data Preprocessing

In [115]:
# split independent and dependent variables
x = dataset.loc[:, dataset.columns != "Price"]
y = dataset["Price"]

In [ ]:
x.head()

In [ ]:
print("Dimension of features = ", x.shape)
print("Dimension of target = ", y.shape)

Split train and test set

In [ ]:
# split train and test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

print("X-train dim: ", x_train.shape)
print("Y-train: ", len(y_train))
print("X-test dim: ", x_test.shape)
print("Y-test: ", len(y_test))

<center> Always: split the data into training and test set, then apply preprocessing!!! <center>

Transforming the numerical and categorical data

In [ ]:
# take metadata
metadata, numeric_cols, categ_cols = get_metadata(x)

<center><b>Criteria to scale numerical features<b> </center>   

**Standard Scaler**  

$$X' = \frac{X - \mu}{\sigma}$$
  
where:  
  
* $X$ is the original feature value  
* $X'$ is the scaled feature value  
* $\mu$ is the mean of the feature values  
* $\sigma$ is the standard deviation of the feature values  
  
**Robust Scaler**  
  
$$X' = \frac{X - Q_1}{Q_3 - Q_1}$$  
  
where:  
  
* $X$ is the original feature value  
* $X'$ is the scaled feature value  
* $Q_1$ is the first quartile (25th percentile) of the feature values  
* $Q_3$ is the third quartile (75th percentile) of the feature values  
  
Note: The Robust Scaler uses the interquartile range (IQR) instead of the standard deviation to make it more robust to outliers.


In [129]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

transformer = make_column_transformer(
    (StandardScaler(), ["Distance", "Lattitude", "Longtitude", "Propertycount"]),  
    (RobustScaler(), ["Rooms", "Postcode", "Bedroom2", "Bathroom", "Car", "Landsize"]),
    (OneHotEncoder(handle_unknown="ignore"), ["SaleDayOfWeek", "Type", "Method", "Regionname"]),
    (OrdinalEncoder(
        categories = [x.Suburb.unique(), x.CouncilArea.unique()], dtype = np.int64), 
        ["Suburb", "CouncilArea"]
    )
)

In [ ]:
# transformer will learn only from training data
transformer.fit(x_train)

In [131]:
# transformer will transform the train and test data
x_train = transformer.transform(x_train)
x_test = transformer.transform(x_test)

In [ ]:
x_train

In [ ]:
y_train

#### 5. Building Model

Benchmark for models:

* XGBoost Regressor
* LightGBM

In [139]:
from sklearn.model_selection import GridSearchCV
import time

In [140]:
import pickle

# function to save model
def save_model(filename, model):
    with open(filename, "wb") as file:
        pickle.dump(model, file)

In [141]:
# function to load model
def load_model(filename):
    with open(filename, "rb") as file:
        return pickle.load(file)

<center> <b>Criteria to evaluate quality of model<b> </center>  
  
**RMSE (Root Mean Squared Error)**
$$\text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}$$

where:

* $y_i$ is the actual value of the $i^{th}$ observation
* $\hat{y}_i$ is the predicted value of the $i^{th}$ observation
* $n$ is the total number of observations

**R2 (Coefficient of Determination)**
$$R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}$$

where:

* $y_i$ is the actual value of the $i^{th}$ observation
* $\hat{y}_i$ is the predicted value of the $i^{th}$ observation
* $\bar{y}$ is the mean of the actual values
* $n$ is the total number of observations


In [1]:
# function to evaluate model
from sklearn.metrics import mean_squared_error, r2_score

def eval_model_perform(model, x, y):    
    y_pred = model.predict(x)
    rmse_val = np.sqrt(mean_squared_error(y, y_pred))
    r2_val = r2_score(y, y_pred)

    return rmse_val, r2_val

**5.1. XGBoost Regressor**

Model Definition

In [158]:
import xgboost as xgb

# hyperparameters definition
xgb_params = {          
    "max_depth": [8, 16, 32],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.7, 0.8],
    "colsample_bytree": [0.8, 0.9],
    "tree_method": ["hist"],
    "objective": ["reg:squarederror"]
}

In [159]:
# define XGB Model
def XGBModel(x_train, y_train, params):
    # define the model    
    model = xgb.XGBRegressor()
    
    # hyperparameter optimization
    grid_search = GridSearchCV(estimator = model,
                               param_grid = params,
                               scoring = "neg_mean_squared_error",
                               cv = 5,
                               n_jobs = -1
                            )
    grid_search.fit(x_train, y_train)
    
    # get best model
    best_model = grid_search.best_estimator_
    print(grid_search.best_params_)
    
    return best_model

Training process

In [160]:
sttrain_xgb = time.time()

In [ ]:
xgb_model = XGBModel(x_train, y_train, xgb_params)

In [ ]:
ettrain_xgb = time.time()
ttrain_xgb = ettrain_xgb - sttrain_xgb
print(f"Time of training: {ttrain_xgb:.3f} seconds")

In [ ]:
xgb_model

In [164]:
# save xgb model
save_model("models/xgb_v1.pkl", xgb_model)

Evaluation process

In [165]:
# recover the model
rec_xgb = load_model("models/xgb_v1.pkl")

In [ ]:
# metrics for train set
rmse_xgb_train, r2_xgb_train = eval_model_perform(rec_xgb, x_train, y_train)
print(f"R-MSE train score: {rmse_xgb_train:.3f}")
print(f"R^2 train score: {r2_xgb_train:.3f}")

In [ ]:
# r2-score for test set
rmse_xgb_test, r2_xgb_test = eval_model_perform(rec_xgb, x_test, y_test)
print(f"R-MSE test score: {rmse_xgb_test:.3f}")
print(f"R^2 test score: {r2_xgb_test:.3f}")

**5.2. Light Gradient Boost Machine (LightGBM)**

Model Definition

In [148]:
# import packages
import lightgbm as lgbm

# define grid hyperparameters
lgbm_params = {    
    "num_leaves": [64, 128, 256],
    "max_depth": [10, 20, 40],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.8, 0.9],
    "subsample_freq": [10] # re-sample without replacement every 10 iterations
                         # and extract bagging_fraction% of training data
}

In [149]:
# define the LightGBM regressor
def LightGBModel(x_train, y_train, params):
    lgbm_model = lgbm.LGBMRegressor()
    
    # hyperparameter optimization
    grid_lgbm = GridSearchCV(estimator = lgbm_model,  # regressor model
                         param_grid = params,  # dict of hyperparameters
                         cv = 5,   # 5-fold cross-validation
                         scoring = "r2",
                         verbose = False,
                         n_jobs = -1
                    )
    # fit the model
    grid_lgbm.fit(x_train, y_train)
    
    # take best model
    best_model = grid_lgbm.best_estimator_
    print(grid_lgbm.best_params_)

    return best_model

Training process

In [150]:
sttrain_lgbm = time.time()

In [ ]:
# take the best model
lgbm_model = LightGBModel(x_train, y_train, lgbm_params)

In [ ]:
ettrain_lgbm = time.time()
ttrain_lgbm = ettrain_lgbm - sttrain_lgbm
print(f"Time of training: {ttrain_lgbm:.3f}")

In [ ]:
lgbm_model

In [153]:
# save xgb model
save_model("models/lgbm_v1.pkl", lgbm_model)

Evaluation process

In [154]:
# recover the model
rec_lgbm = load_model("models/lgbm_v1.pkl")

In [ ]:
# metrics for train set
rmse_lgbm_train, r2_lgbm_train = eval_model_perform(rec_lgbm, x_train, y_train)
print(f"R-MSE train score: {rmse_lgbm_train:.3f}")
print(f"R^2 train score: {r2_lgbm_train:.3f}")

In [ ]:
# r2-score for test set
rmse_lgbm_test, r2_lgbm_test = eval_model_perform(rec_lgbm, x_test, y_test)
print(f"R-MSE test score: {rmse_lgbm_test:.3f}")
print(f"R^2 test score: {r2_lgbm_test:.3f}")

#### 6. Monitoring results

In [171]:
dict_res = {
    "xgboost": pd.DataFrame({
        "train": {"rmse": rmse_xgb_train, "r2": r2_xgb_train},
        "test": {"rmse": rmse_xgb_test, "r2": r2_xgb_test}        
    }),
    "lgbm": pd.DataFrame({
        "train": {"rmse": rmse_lgbm_train, "r2": r2_lgbm_train},
        "test": {"rmse": rmse_lgbm_test, "r2": r2_lgbm_test}        
    })
}

In [ ]:
for key, res in dict_res.items():
    print(f"\nModel: {key}")
    print(res)

In [173]:
import seaborn as sns

def plot_reg_results(res):
    # Create a figure with two subplots
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    # Iterate over the dictionary and plot the results for each model
    sns.set_theme(style="whitegrid")
    for i, (key, res) in enumerate(dict_res.items()):
        # Plot the RMSE values
        sns.barplot(x=[f"{key} Train", f"{key} Test"], y=[res["train"]["rmse"], res["test"]["rmse"]], ax=ax[0])
        # Plot the R2 values
        sns.barplot(x=[f"{key} Train", f"{key} Test"], y=[res["train"]["r2"], res["test"]["r2"]], ax=ax[1])

    # Set the titles and labels for the subplots
    ax[0].set_title("RMSE")
    ax[0].set_xlabel("Model")
    ax[0].set_ylabel("RMSE")

    ax[1].set_title("R2")
    ax[1].set_xlabel("Model")
    ax[1].set_ylabel("R2")

    # Show the plot
    plt.show()


In [ ]:
# monitoring the results
plot_reg_results(dict_res)